# Venus Atmospheric Modelling - Variable Entry Angles
The Open University<br>
Module SXS841 Masters level project in physical sciences<br>
***
End of Module Assignment (EMA) 01<br>
Stuart Gavin Smith<br>
H1872126<br>
25 May 2025<br>
Version:1.0<br>
***

In [3]:
%%capture cap
#Comment out the above line to view text in the output

import io

import numpy as np
import matplotlib.pyplot as plt
from pandas import *
from scipy import interpolate
from scipy.interpolate import interp1d
import datetime

#Ignore deprecated function on data frame append
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

#Use the data created in the atmospheric modelling 
ATM_CSV = r'C:\Users\stuar\OneDrive\Open University\SXS841\Venus Atmospheric Modelling\CSV_Exports\dfALT_D.csv'
ATP_CSV = r'C:\Users\stuar\OneDrive\Open University\SXS841\Venus Atmospheric Modelling\CSV_Exports\dfALT_T.csv'

#Read CSV data into dataframe, on error, print warning and stop kernel.
try:
    dfA=read_csv(ATM_CSV, skiprows=1,names=['ALT-KM','RHO-KG-M3'])
except:
    print("ATM_CSV Venus data does not exist, or some other error")
    Error = True
    exit()

#Read CSV data into dataframe, on error, print warning and stop kernel.
try:
    dfT=read_csv(ATP_CSV, skiprows=1,names=['ALT-KM','T-DEG-K','T-DEG-C'])
except:
    print("ATP_CSV Venus data does not exist, or some other error")
    Error = True
    exit()

#scale and force convert CSV data to float
dfA['ALT-KM'] = dfA['ALT-KM'] * 1000.0
dfA['RHO-KG-M3'] = dfA['RHO-KG-M3'] * 1000.0

dfT['ALT-KM'] = dfT['ALT-KM'] * 1000.0
dfT['T-DEG-K'] = dfT['T-DEG-K'] 
dfT['T-DEG-C'] = dfT['T-DEG-C'] 

#Sort original data for plotting
dfA.sort_values("ALT-KM", ascending=True)
dfT.sort_values("ALT-KM", ascending=True)

#Model Constants
maxmetres = 170000 # model start altitude, 10km above interface

# Constants - Venus
g    = 8.87    # Venus Gravity m/s^2
R    = 6051800 # equatorial mean radius
RHOs = 65      # Atmospheric density at sea level (kg/m^3)
Hs   = 15819   # atmospheric scale height

#Constants - Vehicle and Probe
Rnose = 1 # 1m radius of nose cone

CD  = 1.05  # Drag coefficient for a blunted cone
CDd = 0.55  # Drag coefficient for a drogue chute
CDs = 0.47  # Drag coefficient for a spherical shape
CDp = 1.75  # Drag coefficient for a typical parachute
CDb = 0.47  # Drag coefficient for a baloon, assume spherical inflation from centre of sphere (simplified)

S  = np.pi*(1.0*1.0)   # Cross-sectional area (m^2) of 2.0m diameter entry capsule (and heat shield)
Sd = np.pi*(1.5*1.5)   # Cross-sectional area (m^2) of 3.0m diameter drogue chute
Sp = np.pi*(4.25*4.25) # Cross-sectional area (m^2) of 8.5m diameter main parachute
Sb = np.pi*(3.5*3.5)   # Cross-sectional area (m^2) of 7.0m diameter balloon

mPayloadAndBalloonAndGas = 203.31 + (214.50 / g) + 35.79
mGasCylinders            = 72 * 13.84
mBackShell               = 50.0
mHeatShield              = 200.0
mDrogueChute             = 25.0
mMainParachute           = 50.0

# Initial mass
m  = mPayloadAndBalloonAndGas + mGasCylinders + mBackShell + mHeatShield + mDrogueChute + mMainParachute

#Constants - flight path
y0 = 15         # entry angle in degrees
Ve = 7816       # entry velocity m/s from the orbital speed of 160km circular orbit
h  = 160000     # m atmospheric interface
r  = R + h/1000 # (km)

#Inflate balloon when the horizontal velocity reaches the predicted windspeed of 60-70 m/s 
#for a lower cloud base altitude (47 km) - from Sánchez-Lavega, A., Hueso, R., Piccioni, G., 
#Drossart, P., Peralta, J., Pérez-Hoyos, S., Wilson, C. F., Taylor, F. W., Baines, K. H., 
#Luz, D., Erard, S., & Lebonnois, S. (2008). Variable winds on Venus mapped in three 
#dimensions. Geophysical Research Letters, 35(13). https://doi.org/10.1029/2008GL033817
HorizontalVelocityMax = 65.0 # m/s

#Maximum number of model step iterations
aMaxMS = 30000000
  
#Set the step resoluton and initial altitude     
StepResolution = 1000.0 # 1000 for milliseconds, 1 for seconds

print("==============================SIMULATION CONFIGURATION=================================")
print("")
print("= Drag Co-Efficients and Cross Sectional Area")
print("")
print("Probe CD (coef.drag)  : " + f'{CD:.4f}' )
print("Probe S (X section)   : " + f'{S:.4f}' + str(' m2'))
print("")
print("= Mass")
print("")
print("Probe Mass            : " + f'{m:.2f}' + str(' kg'))
print("")
print("= Ballistic Entry Characteristics")
print("")
print("Entry Angle           : VARIABLE")
print("Entry Velocity        : " + f'{Ve:.2f}' + str(' ms'))
print("Start Altitude        : " + f'{Ve:.2f}' + str(' maxmetres'))
print("")
print("= Atmospheric and Planetary")
print("")
print("Atmosphere Interface  : " + f'{h:.2f}' + str(' m'))
print("Scale Height          : " + f'{Hs:.2f}' + str(' m'))
print("Gravity (Venus)       : " + f'{g:.2f}' + str(' ms/s'))
print("Equatorial Mean Radius: " + f'{R:.2f}' + str(' ms/s'))
print("")
print("= Simulation Events")
print("")
print("Max Simluation Steps  : " + f'{aMaxMS:.2f}' + str(' steps'))
print("Step Resolution       : " + f'{StepResolution:.2f}' + str(' [1000 = ms / 1 = seconds]'))
print("")


#Arrays to hold the atmospheric measurement models of Density and Temperature
densityX = np.linspace(0,maxmetres,maxmetres)
densityY = np.arange(maxmetres,dtype=float)
tempX = np.linspace(0,maxmetres,maxmetres)
tempY = np.arange(maxmetres,dtype=float)
densityY2 = np.arange(maxmetres,dtype=float)

# Function to find the nearest density value in the dataframe
def find_nearest_density(df, AltCol, DenCol, value):
    arrayalt = df[AltCol].values
    arrayrho = df[DenCol].values
    idx = (np.abs(arrayalt - (1.0*value))).argmin()
    return arrayrho[idx]

# Function to find the nearest temperature value in the dataframe
def find_nearest_temp(df, AltCol, TempCol, value):
    arrayalt = df[AltCol].values
    arraydegC = df[TempCol].values
    idx = (np.abs(arrayalt - (1.0*value))).argmin()
    return arraydegC[idx]

# Create a 1m density height model from measurements
for AltCount in range(maxmetres):
    nearest_density = find_nearest_density(dfA, 'ALT-KM', 'RHO-KG-M3',AltCount)
    densityY[AltCount] = nearest_density

# Create a 1m temperature height model from measurements
for AltCount in range(maxmetres):
    nearest_temp = find_nearest_temp(dfT, 'ALT-KM', 'T-DEG-C',AltCount)
    tempY[AltCount] = nearest_temp
    
#From mathematical means, create an atmosphere model for validation purposes
for AltCount in range(maxmetres):
      #ρ = ρ0 exp ( -h / H )
      densityY2[AltCount] = RHOs * np.exp(-(AltCount/Hs))

    
# Interpolate the 'step' changes to increase fidelity to 1m steps    
Y1 = densityY[0]
Y2 = 0.0
X1 = 0
X2 = 0

#For each metre of height, smooth (simple interpolation)
for AltCount in range(maxmetres):
    if Y1 != densityY[AltCount]:
        #A step change has been detected
        X2 = AltCount - 1
        Y2 = densityY[AltCount]

        #Calculate the scaling factor to apply (interpolate)
        Yfactor = (Y2-Y1)/(X2-X1+1)
        
        #Now interpolate between the 2 points
        for InterpCount in range(X2-X1+1):
            densityY[int(X1+InterpCount)] = Y1 + (Yfactor*InterpCount)
        
        #Start a new step 
        Y1 = densityY[AltCount]
        X1 = AltCount

# Interpolate the 'step' changes to increase fidelity to 1m steps    
Y1 = tempY[0]
Y2 = 0.0
X1 = 0
X2 = 0

#For each metre of height, smooth (simple interpolation so each metre of height has a value)
for AltCount in range(maxmetres):
    if Y1 != tempY[AltCount]:
        #A step change has been detected
        X2 = AltCount - 1
        Y2 = tempY[AltCount]

        #Calculate the scaling factor to apply (interpolate)
        Yfactor = (Y2-Y1)/(X2-X1+1)
        
        #Now interpolate between the 2 points
        for InterpCount in range(X2-X1+1):
            tempY[int(X1+InterpCount)] = Y1 + (Yfactor*InterpCount)
        
        #Start a new step 
        Y1 = tempY[AltCount]
        X1 = AltCount    

        
def StepModel(bUseMeasurements, EntryAngle, tempY, densityY, tempX, densityX, densityY2):

    #Entry Angle
    y0 = EntryAngle
    #Atmospheric Model or Measurement Basis
    UseMeasurements = bUseMeasurements

    
    #Set the 1st step conditions
    Alt = maxmetres
    Vcurrent = Ve
    aStep = 0.0
    Acc = 0.0
    Drag = 0.0
    AngleD = 0.0
    VerticalD = 0.0
    VerticalDstep = 0.0
    HorizontalD = 0.0
    VerticalV = 0.0
    HorizontalV = 0.0
    RHOmd = 0.0
    Tempmd = 0.0

    #Set the flags
    DrogueChuteDeployed = False
    DrogueChuteInflated = False
    ParachuteDeployed = False
    ParachuteInflated = False
    BalloonDeployed = False
    BalloonInflated = False
    GravityDominant = False
    DragisZero = False
    GravGreaterThanDrag = False

    #Convert entry angle to radians
    gammaR = np.radians(y0)

    #Step model data frame
    dfStepModel = DataFrame(columns=['TimeMS','Alt','Acc','Drag','Velocity','VerticalV','HorizontalV','HorizontalD', 'VerticalD', 'AngleD', 'Density', 'Temperature'])

    #Peak Values and Time Markers
    PeakAcc = 0.0
    PeakAccTimeStep = 0.0
    PeakTemp = -273.15
    PeakTempTimeStep = 0.0
    PeakDensity = 0.0
    PeakDensityTimeStep = 0.0
    LowestAltitude = Alt
    LowestAltitudeTimeStep = 0.0

    print("==================================SIMULATION START=====================================")
    StartTimeDate = datetime.datetime.now()

    #Create the step model
    for msCount in range(aMaxMS):

        #Exit if we hit the ground !
        if Alt <= 0:
            print(f'{msCount/StepResolution:08.3f}'+ ' Hit ground, model stopping')        
            Alt = 0
            break;

        #Exit if velocities go below ZERO past the atmospheric interface
        if Vcurrent <= 0 and Alt <= h:
            #Velocity along flight path has been 'scrubbed' to zero by the drag forces, now the balloon 
            #if inflated will start to rise, and the vertical and horizontal velocities will only be impacted
            #by lift forces and wind speed 
            print(f'{msCount/StepResolution:08.3f}'+ ' Negative Velocity Vcurrent')
            print(f'{msCount/StepResolution:08.3f}'+ ' Velocity Vcurrent: ' + str(Vcurrent))
            print(f'{msCount/StepResolution:08.3f}'+ ' Velocity Vvert: ' + str(VerticalV))
            print(f'{msCount/StepResolution:08.3f}'+ ' Velocity Vhoriz: ' + str(HorizonatalV))
            Alt = 0
            break;
        if VerticalV <= 0 and Alt <= h:
            print(f'{msCount/StepResolution:08.3f}'+ ' Negative Velocity Vvert')
            print(f'{msCount/StepResolution:08.3f}'+ ' Velocity Vcurrent: ' + str(Vcurrent))
            print(f'{msCount/StepResolution:08.3f}'+ ' Velocity Vvert: ' + str(VerticalV))
            print(f'{msCount/StepResolution:08.3f}'+ ' Velocity Vhoriz: ' + str(HorizonatalV))
            Alt = 0
            break;
        if HorizontalV <= 0 and Alt <= h:
            print(f'{msCount/StepResolution:08.3f}'+ ' Negative Velocity Vhoriz')
            print(f'{msCount/StepResolution:08.3f}'+ ' Velocity Vcurrent: ' + str(Vcurrent))
            print(f'{msCount/StepResolution:08.3f}'+ ' Velocity Vvert: ' + str(VerticalV))
            print(f'{msCount/StepResolution:08.3f}'+ ' Velocity Vhoriz: ' + str(HorizonatalV))
            Alt = 0
            break;

        #calculate the velocity and displacement in the horizontal and vertical at end of step  
        AngleD = AngleD + (Vcurrent / StepResolution)
        VerticalDstep = abs((np.sin(gammaR) * (Vcurrent / StepResolution)))
        VerticalD = VerticalD + abs((np.sin(gammaR) * (Vcurrent / StepResolution)))
        HorizontalD = HorizontalD + (np.cos(gammaR) * (Vcurrent / StepResolution))
        VerticalV = np.abs(np.sin(gammaR) * Vcurrent)
        HorizontalV = np.abs(np.cos(gammaR) * Vcurrent)

        #Get the density of air for this altitude using measurements or calculate from equation         
        if UseMeasurements == True:
            RHOmd = densityY[int(Alt)-1]
        else:
            RHOmd = densityY2[int(Alt)-1]
            
           
        #Get the temperature for this altitude
        Tempmd = tempY[int(Alt)-1]

        #Calulate the drag force of the main probe (without drogue or main chute) - assume constant CD and S 
        Drag = 0.5 * RHOmd * ((Vcurrent)*(Vcurrent)) * CD * S

        #Calculate gravitational force (at angle) acting on the trajectory velocity along path of entry
        GravForce = m * (g / np.sin(gammaR))

        if GravForce > Drag and Alt < 40000:
            #Gravity becoming dominant 
            if GravityDominant == False:
                GravityDominant = True
                print(f'{msCount/StepResolution:08.3f}'+ ' Gravity dominant force')

        #Calculate the deceleration (F = ma) adjusting drag for gravity along the path of entry 
        if Drag < GravForce or Drag < 0.0:

            if Drag < 0.0:
                #Drag should not be negative in this model until exited the entry flight path and in 'ascension'
                if DragisZero == False:
                    DragisZero = True;
                    print(f'{msCount/StepResolution:08.3f}'+ ' Drag negative, Gravity Force: ' + f'{GravForce:.2f}' + ' N')
                Acc = GravForce / m
            else:
                if GravGreaterThanDrag == False and Alt < 50000:
                    GravGreaterThanDrag = True
                    print(f'{msCount/StepResolution:08.3f}'+ ' Gravity > Drag')
                    print(f'{msCount/StepResolution:08.3f}'+ ' Drag Force: ' + f'{Drag:.2f}' + ' N')
                    print(f'{msCount/StepResolution:08.3f}'+ ' Gravity Force: ' + f'{GravForce:.2f}' + ' N')
                Acc = (GravForce - Drag) / m
        else:
            #Acceleration adjusted for effect of gravity 
            Acc = (Drag - GravForce) / m

        #Calculate the exit altitude
        Alt = Alt - VerticalDstep

        #Calculate the exit velocity along the path of entry of this step, decceleration acts over 1m-ms/1m-ms
        Vcurrent = Vcurrent - abs(Acc / StepResolution / StepResolution)

        #Store the step output states
        stepData = {'TimeMS':msCount,
                    'Alt' : Alt,
                    'Acc' : Acc,
                    'Drag' : Drag,
                    'Velocity' : Vcurrent,
                    'VerticalV' : VerticalV,
                    'HorizontalV' : HorizontalV,
                    'HorizontalD' : HorizontalD,
                    'VerticalD' : VerticalD,
                    'AngleD' : AngleD,
                    'Density' : RHOmd,
                    'Temperature' : Tempmd}
        dfStepModel = dfStepModel.append(stepData, ignore_index=True)

        #Store peak values and timestamps
        if PeakAcc < Acc:
            PeakAcc = Acc
            PeakAccTimeStep = msCount

        if PeakTemp < Tempmd:
            PeakTemp = Tempmd
            PeakTempTimeStep = msCount

        if PeakDensity < RHOmd:
            PeakDensity = RHOmd
            PeakDensityTimeStep = msCount

        if LowestAltitude > Alt:
            LowestAltitude = Alt
            LowestAltitudeTimeStep = msCount

        #Comfort Message whilst processing (10 seconds)
        if msCount % (10 * StepResolution) == 0:
            print(f'{msCount/StepResolution:08.3f}'+ ' Reached altitude of ' + f'{Alt/1000:.2f}' + ' km')

        #Stop the model when velocity has fallen off to around windspeed
        if Vcurrent < 400:
            #Exit the Loop 
            break;

        #End of Loop

    #Mark end of simulation
    EndTimeDate = datetime.datetime.now()
    TimeDifference = EndTimeDate - StartTimeDate
    RunTimeSeconds = TimeDifference.total_seconds()

    #Print summary
    print("")
    print("==================================SIMULATION ENDED=====================================")
    print("")

    print("====================================SUMMARY DATA=======================================")
    print("Start Time (real)     : " + str(StartTimeDate))
    print("End Time (real)       : " + str(EndTimeDate))
    print("Run Time (real)       : " + str(RunTimeSeconds) + str(' seconds'))
    print("Run Time (sim)        : " + f'{msCount/StepResolution:.2f}' + str(' seconds'))
    print("Velocity              : " + f'{Vcurrent:.2f}' + str(' m/s'))
    print("Acceleration          : " + f'{Acc / StepResolution / StepResolution:.2f}' + str(' m/s/s'))
    print("Vertical Velocity     : " + f'{VerticalV:.2f}' + str(' m/s'))
    print("Horizontal Velocity   : " + f'{HorizontalV:.2f}' + str(' m/s'))
    print("Vertical Distance     : " + f'{VerticalD / 1000.0:.2f}' + str(' km'))
    print("Horizontal Distance   : " + f'{HorizontalD / 1000.0:.2f}' + str(' km'))
    print("Max Deceleration      : " + f'{PeakAcc / g / StepResolution:.2f}' + str(' g'))
    print("Max Deceleration Time : " + f'{PeakAccTimeStep/StepResolution:.2f}' + str(' seconds'))
    print("Max Temperature       : " + f'{PeakTemp:.2f}' + str(' Degrees C'))
    print("Max Temperature Time  : " + f'{PeakTempTimeStep/StepResolution:.2f}' + str(' seconds'))
    print("Max Density           : " + f'{PeakDensity:.4f}' + str(' kg/m3'))
    print("Max Density Time      : " + f'{PeakDensityTimeStep/StepResolution:.2f}' + str(' seconds'))
    print("Lowest Altitude       : " + f'{LowestAltitude / 1000.0:.2f}' + str(' km'))
    print("Lowest Altitude Time  : " + f'{LowestAltitudeTimeStep/StepResolution:.2f}' + str(' seconds'))
    print("=======================================================================================")

    print("")
    print("=========================================PLOT==========================================")
    print("")    

    #Plot the acceleration in g in seconds
    dfStepModel['TimeMS'] = dfStepModel['TimeMS'] / 1000
    dfStepModel['Acc'] = dfStepModel['Acc'] / g / StepResolution
    plt.scatter(dfStepModel['TimeMS'],dfStepModel['Acc'], s=0.2, linewidth=0)
    if UseMeasurements == True:
        plt.title('Ballistic Entry - (De)Acceleration Measured: ' + str(y0))
    else:
        plt.title('Ballistic Entry - (De)Acceleration Numerical: ' + str(y0))
    plt.xlabel('Time (s)')
    plt.ylabel('Deceleration (g)')
    plt.grid(True)
    plt.savefig(r'C:\Users\stuar\OneDrive\Open University\SXS841\Venus Entry Modelling\EntryAngle_' + str(y0) + str(UseMeasurements) + '.png',dpi=300, bbox_inches='tight') 
    plt.show()
    
    # Max Acceleration (Validation) - numerical only
    if UseMeasurements == False:
        Beta = 1/Hs 
        amax = -((Beta*Ve*Ve)/(2*np.exp(1))*np.sin(gammaR))
        print("Numerical Acc. Peak   : " + f'{amax:.2f}' + ' m/s/s')
        print("Numerical Acc. Peak g : " + f'{amax / g:.2f}' + ' g')


    #Store Deceleration Data
    dfAlt_DecStore = dfStepModel[['TimeMS','Acc']]
    dfAlt_DecStore .to_csv(r'C:\Users\stuar\OneDrive\Open University\SXS841\Venus Entry Modelling\EntryAngle_' + str(y0) + str(UseMeasurements) + '.csv', encoding='utf-8', index=False)

    print("")
    print("====================================END OF SCRIPT======================================")
    print("") 

    return dfAlt_DecStore 
    #End of Step Model Function


#Run the step models - with measurements
print("")
print("====================================Call Step T15======================================")
print("")   
plotDataT15 = StepModel(True,15, tempY, densityY, tempX, densityX, densityY2)
print("")
print("====================================Call Step T30======================================")
print("")   
plotDataT30 = StepModel(True,30, tempY, densityY, tempX, densityX, densityY2)
print("")
print("====================================Call Step T45======================================")
print("")   
plotDataT45 = StepModel(True,45, tempY, densityY, tempX, densityX, densityY2)
print("")
print("====================================Call Step T60======================================")
print("")   
plotDataT60 = StepModel(True,60, tempY, densityY, tempX, densityX, densityY2)
print("")
print("====================================Call Step T75======================================")
print("")   
plotDataT75 = StepModel(True,75, tempY, densityY, tempX, densityX, densityY2)

#Run the step models - using atmospheric scale model
print("")
print("====================================Call Step F15======================================")
print("")   
plotDataF15 = StepModel(False,15, tempY, densityY, tempX, densityX, densityY2)
print("")
print("====================================Call Step F30======================================")
print("")   
plotDataF30 = StepModel(False,30, tempY, densityY, tempX, densityX, densityY2)
print("")
print("====================================Call Step F45======================================")
print("")   
plotDataF45 = StepModel(False,45, tempY, densityY, tempX, densityX, densityY2)
print("")
print("====================================Call Step F60======================================")
print("")   
plotDataF60 = StepModel(False,60, tempY, densityY, tempX, densityX, densityY2)
print("")
print("====================================Call Step F75======================================")
print("")   
plotDataF75 = StepModel(False,75, tempY, densityY, tempX, densityX, densityY2)


 #Overlay Plot all the graphs
plt.figure(figsize=(12, 4))

plt.scatter(plotDataT15['TimeMS'],plotDataT15['Acc'], s=0.2, linewidth=0, label='15 degrees (M)')
plt.scatter(plotDataT30['TimeMS'],plotDataT30['Acc'], s=0.2, linewidth=0, label='30 degrees (M)')
plt.scatter(plotDataT45['TimeMS'],plotDataT45['Acc'], s=0.2, linewidth=0, label='45 degrees (M)')
plt.scatter(plotDataT60['TimeMS'],plotDataT60['Acc'], s=0.2, linewidth=0, label='60 degrees (M)')
plt.scatter(plotDataT75['TimeMS'],plotDataT75['Acc'], s=0.2, linewidth=0, label='75 degrees (M)')
plt.scatter(plotDataF15['TimeMS'],plotDataF15['Acc'], s=0.2, linewidth=0, label='15 degrees (S)')
plt.scatter(plotDataF30['TimeMS'],plotDataF30['Acc'], s=0.2, linewidth=0, label='30 degrees (S)')
plt.scatter(plotDataF45['TimeMS'],plotDataF45['Acc'], s=0.2, linewidth=0, label='45 degrees (S)')
plt.scatter(plotDataF60['TimeMS'],plotDataF60['Acc'], s=0.2, linewidth=0, label='60 degrees (S)')
plt.scatter(plotDataF75['TimeMS'],plotDataF75['Acc'], s=0.2, linewidth=0, label='75 degrees (S)')

plt.xlabel('Time (s)')
plt.ylabel('Deceleration (g)')

lgnd = plt.legend(loc='upper right', title="Entry Angles", scatterpoints=1, fontsize=10)
lgnd.legendHandles[0]._sizes = [30]
lgnd.legendHandles[1]._sizes = [30]
lgnd.legendHandles[2]._sizes = [30]
lgnd.legendHandles[3]._sizes = [30]
lgnd.legendHandles[4]._sizes = [30]
lgnd.legendHandles[5]._sizes = [30]
lgnd.legendHandles[6]._sizes = [30]
lgnd.legendHandles[7]._sizes = [30]
lgnd.legendHandles[8]._sizes = [30]
lgnd.legendHandles[9]._sizes = [30]

plt.grid(True)

plt.savefig(r'C:\Users\stuar\OneDrive\Open University\SXS841\Venus Entry Modelling\EntryAnglesCombined.png',dpi=300, bbox_inches='tight') 

plt.show()




In [4]:
#Store the output text
with open(r'C:\Users\stuar\OneDrive\Open University\SXS841\Venus Entry Modelling\EntryAngleModelling.txt', 'w') as file:
    file.write(cap.stdout)

